In [1]:
import requests
from bs4 import BeautifulSoup
import openpyxl
from openpyxl.styles import Font
import re

wb = openpyxl.Workbook() # Создание новой книги Excel
ws = wb.active # Присваивание активного листа

headers = ['Название', 'Артикул', 'Крепость', 'Объем (л)', 'Цвет', 'Сахар', 
           'Бренд', 'Страна', 'Регион', 'Субрегион', 'Выдержка в ёмкости',
           'Виноград', 'Температура подачи', 'Потенциал хранения', 'Стиль вина', 'Гастрономия',
           'Описание', 'Подробнее', 'Винтаж', 'Производитель', 'Категория', 'Выдержка', 'Отзыв']

dict_headers = {'Название': 'A', 'Артикул': 'B', 'Крепость': 'C', 'Объем (л)': 'D', 'Цвет': 'E', 'Сахар': 'F', 
           'Бренд': 'G', 'Страна': 'H', 'Регион': 'I', 'Субрегион': 'J', 'Выдержка в ёмкости': 'K',
           'Виноград': 'L', 'Температура подачи': 'M', 'Потенциал хранения': 'N', 'Стиль вина': 'O', 'Гастрономия': 'P',
           'Описание': 'Q', 'Подробнее': 'R', 'Винтаж': 'S', 'Производитель': 'T',  'Категория': 'U', 'Выдержка': 'V', 'Отзыв': 'W'}

ws.append(headers)

bold_font = Font(bold=True) # Жирный шрифт для заголовков

for cell in ws[1]:
    cell.font = bold_font

base_url = "https://spb.decanter.ru"

catalog_url = base_url + "/wine/reviews?PAGEN_1="

response_page = requests.get(base_url)

soup_page = BeautifulSoup(response_page.text, 'html.parser')

count_row = 2 # Счетчик строк

for page in range(1, 55):
    print(page)
    
    page_url = catalog_url + f"{page}"

    response_page = requests.get(page_url)

    if response_page.status_code != 200:
        print("Ошибка при запросе к странице каталога")
        exit()

    soup_page = BeautifulSoup(response_page.text, 'html.parser')

    
    a_list = soup_page.find_all('a', class_="prod-review__link", href = True)

    a_list_info = [info_link for info_link in a_list[1::2]] # Все нечетные элементы

    for elem_a in a_list_info:
        url_rev = base_url + elem_a['href']
        url_wine = url_rev[:-8]

        
        response_wine = requests.get(url_wine)

        if response_wine.status_code != 200:
            print("Ошибка при запросе к странице каталога с вином")
            exit()

        soup_wine = BeautifulSoup(response_wine.text, 'html.parser')

        response_rev = requests.get(url_rev)

        if response_rev.status_code != 200:
            print("Ошибка при запросе к странице каталога с отзывами")
            exit()

        soup_rev = BeautifulSoup(response_rev.text, 'html.parser')



        try:
            # Находим количество отзывов
            rev_count = soup_rev.find('p', class_ = 'prod-reviews__count').get_text(strip=True)
            rev_count = int(re.findall(r'\d+', rev_count)[0])

            rev_text_list = soup_rev.find_all('div', class_ = 'fb-card__text')

        
            for rev_text in rev_text_list:
                cell_address = f'{dict_headers["Отзыв"]}{count_row}'
                ws[cell_address] = rev_text.get_text(strip=True)
                count_row += 1

            # Парсинг характеристик вина

            ch_list = soup_wine.find_all('span', class_ = 'prod-card__info-title')

            for elem_info in ch_list:

                title_ch = elem_info.get_text(strip=True)

                if title_ch in ['Цвет', 'Сахар', 'Бренд', 'Страна', 'Регион', 'Субрегион',
                                'Выдержка в ёмкости', 'Виноград', 'Стиль вина', 'Подробнее', 'Винтаж', 'Производитель']:
                    
                    value_ch = elem_info.find_next('a', href = True).get_text(strip=True)

                else:
                    value_ch = elem_info.find_next('span').get_text(strip=True)


                col_letter = dict_headers[title_ch]

                for count_row_ch in range(count_row - rev_count-1, count_row):

                    # Формируем адрес ячейки
                    cell_address = f'{col_letter}{count_row_ch}'

                    # Устанавливаем значение в ячейку
                    ws[cell_address] = value_ch


            # Гастрономия
            gastronomy_list = soup_wine.find('div', class_ = 'prod-card__tastes-box')
            
            gast_list = []
            try:
                a_gast = gastronomy_list.find_all('a', title = True)
                gast_list = []
                for a_gast_elem in a_gast:
                    gast_list.append(a_gast_elem['title'])
                    gast_str = ', '.join(elem for elem in gast_list)

            except:
                gast_str = ''
            
            for count_row_ch in range(count_row - rev_count-1, count_row):

                # Формируем адрес ячейки
                cell_address = f'{dict_headers["Гастрономия"]}{count_row_ch}'

                # Устанавливаем значение в ячейку
                ws[cell_address] = gast_str

            # Описание
            desk_list = soup_wine.find('div', class_ = 'prod-card__description')
            description = desk_list.get_text()

            for count_row_ch in range(count_row - rev_count-1, count_row):

                # Формируем адрес ячейки
                cell_address = f'{dict_headers["Описание"]}{count_row_ch}'

                # Устанавливаем значение в ячейку
                ws[cell_address] = description

        except:
            exit()

        
wb.save("wine_decanter_rev.xlsx")

print("Парсинг завершен. Данные сохранены в wine_decanter.xlsx")


1
Ошибка при запросе к странице каталога с вином
Ошибка при запросе к странице каталога с отзывами
2
3
Ошибка при запросе к странице каталога с вином
Ошибка при запросе к странице каталога с отзывами
4
Ошибка при запросе к странице каталога с вином
Ошибка при запросе к странице каталога с отзывами
Ошибка при запросе к странице каталога с вином
Ошибка при запросе к странице каталога с отзывами
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
Парсинг завершен. Данные сохранены в wine_decanter.xlsx


: 